In [3]:
import cv2
import os
import numpy as np

labels = ['Negative', 'Positive']
img_size = 100

def read_images(data_dir):
    images = []
    labels_list = []
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE) 
                # Resize the image to 120*120
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) 
                images.append(resized_arr)
                labels_list.append(class_num)
            except Exception as e:
                print(e)
    return np.array(images), np.array(labels_list)

X, y = read_images('/kaggle/input/concrete-crack-images-for-classification/Concrete')

X = X.reshape(-1, img_size, img_size, 1)

# Normalize pixel values to [0, 1]
X = X / 255.0

# Split data into training and validation sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model 1 - CNN w 4 Conv Layers

In [9]:
from keras import layers
from keras import models
from keras import optimizers
from keras import losses
from keras import metrics

from keras.preprocessing.image import img_to_array, load_img

model = models.Sequential([
    layers.Input(shape=(100, 100, 1)),
    layers.Conv2D(32, (3, 3)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [10]:
# model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 98, 98, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 98, 98, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 98, 98, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 49, 49, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 47, 47, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 47, 47, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_5 (Activation)            │ (None, 47, 47, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 23, 23, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 21, 21, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 21, 21, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_6 (Activation)            │ (None, 21, 21, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 10, 10, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 8, 8, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 8, 8, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_7 (Activation)            │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 1,291,265 (4.93 MB)

 Trainable params: 1,290,561 (4.92 MB)

 Non-trainable params: 704 (2.75 KB)

In [13]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - binary_accuracy: 0.9986 - loss: 0.0043 - val_binary_accuracy: 0.9984 - val_loss: 0.0042
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - binary_accuracy: 0.9990 - loss: 0.0034 - val_binary_accuracy: 0.9989 - val_loss: 0.0028
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - binary_accuracy: 0.9992 - loss: 0.0030 - val_binary_accuracy: 0.9987 - val_loss: 0.0040
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - binary_accuracy: 0.9994 - loss: 0.0024 - val_binary_accuracy: 0.9995 - val_loss: 0.0022
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - binary_accuracy: 0.9991 - loss: 0.0030 - val_binary_accuracy: 0.9992 - val_loss: 0.0028
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - binary_accuracy: 0.9994 - loss: 0.0025 - val_binary_accuracy: 0.9966 - val_loss: 0.0125
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - binary_accuracy: 0.9990 - loss: 0.0033 - val_binary_accuracy: 0.9994 - val_loss: 0.0026

In [14]:
from sklearn.metrics import f1_score

prediction = model.predict(X_test)
y_pred = (prediction >= 0.5).astype("int32")

# Calculate F1-Score
f1 = f1_score(y_test, y_pred)
print(f"F1-Score: {f1 * 100:.2f}%")

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
F1-Score: 99.91%
